In [1]:
!git clone https://github.com/Anti-Entrophic/FastSpeechPPG.git
%cd /content/FastSpeechPPG/FastSpeech2
!pip3 install -r requirements.txt
!pip install g2p_en
!pip install inflect
!pip install librosa
!pip install matplotlib
!pip install pyworld
!pip install tgt
!pip install pypinyin==0.39.0
!pip install unidecode
!pip install --upgrade --no-cache-dir gdown

Cloning into 'FastSpeechPPG'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 147 (delta 14), reused 71 (delta 7), pack-reused 52
Receiving objects: 100% (147/147), 8.20 MiB | 14.28 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/FastSpeechPPG/FastSpeech2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 26.5 MB/s eta 0:00:00
  Installi

# Infer

In [ ]:
#貌似不行
!gdown "https://drive.google.com/uc?id=1DBRkALpPd6FL9gjHMmMEdHODmkgNIIK4"

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1DBRkALpPd6FL9gjHMmMEdHODmkgNIIK4 



In [ ]:
!python3 synthesize.py --text "大家好" --speaker_id 1 --restore_step 600000 --mode single -p config/AISHELL3/preprocess.yaml -m config/AISHELL3/model.yaml -t config/AISHELL3/train.yaml

Traceback (most recent call last):
  File "/content/FastSpeech2/synthesize.py", line 188, in <module>
    model = get_model(args, configs, device, train=False)
  File "/content/FastSpeech2/utils/model.py", line 20, in get_model
    ckpt = torch.load(ckpt_path)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 771, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 270, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 251, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: './output/ckpt/AISHELL3/600000.pth.tar'


# Train(原版)

## 准备数据

In [2]:
!pip install wget
!pip install tarfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=bfb823a1a3b5092d0355256a771bdd4daf363f5b22ab19ac0eb09ac3080dee0b
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tarfile (from versions: none)
ERROR: No matching distribution found for tarfile


In [3]:
#@title 下载 LJspeech
# 3min
%cd /content
!wget --no-check-certificate -r http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2

/content
--2023-03-17 15:40:47--  http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2’

data.keithito.com/d 100%[===================>]   2.56G  12.1MB/s    in 3m 23s  

2023-03-17 15:44:11 (12.9 MB/s) - ‘data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]

FINISHED --2023-03-17 15:44:11--
Total wall clock time: 3m 24s
Downloaded: 1 files, 2.6G in 3m 23s (12.9 MB/s)


In [4]:
#@title 解压
# 10min
%cd /content/data.keithito.com/data/speech/
import tarfile
filename = "LJSpeech-1.1.tar.bz2"
tf = tarfile.open(filename)
tf.extractall('/content')

/content/data.keithito.com/data/speech


## 训练

In [9]:
#@title 测试日志模块（不用再跑了）
%cd /content/FastSpeechPPG/FastSpeech2
import logging
logging.basicConfig(level=logging.DEBUG #设置日志输出格式
      ,filename="/content/experiment.log" #log日志输出的文件位置和文件名
      ,format="%(asctime)s-%(levelname)s: %(message)s" #日志输出的格式
                      # -8表示占位符，让输出左对齐，输出长度都为8位
      ,datefmt="%Y-%m-%d %H:%M:%S"  #时间输出的格式
      ,force=True)
# test
logging.debug("debug!")

# 这样用就可以了
from debugging import logging_init
import logging

/content/FastSpeechPPG/FastSpeech2


In [5]:
#@title 预处理，好像是去除文字的大小写和特殊符号。还有对音频处理了一下，看不懂。
# 4min
%cd /content/FastSpeechPPG/FastSpeech2
!python3 prepare_align.py config/LJSpeech/preprocess.yaml

/content/FastSpeechPPG/FastSpeech2
13100it [02:34, 84.74it/s]
Traceback (most recent call last):
  File "/content/FastSpeechPPG/FastSpeech2/prepare_align.py", line 23, in <module>
    main(config)
  File "/content/FastSpeechPPG/FastSpeech2/prepare_align.py", line 10, in main
    ljspeech.prepare_align(config)
  File "/content/FastSpeechPPG/FastSpeech2/preprocessor/ljspeech.py", line 56, in prepare_align
    assert 0
AssertionError


上传 `LJSpeech.zip` 到路径 `FastSpeech2/preprocessed_data/LJSpeech/TextGrid/` 下，并解压

In [6]:
# 解压
%cd /content/FastSpeechPPG/FastSpeech2/preprocessed_data/LJSpeech
import zipfile
import os
assert os.path.exists('./LJSpeech.zip')
zip_file = zipfile.ZipFile('./LJSpeech.zip')
zip_file.extractall()

/content/FastSpeechPPG/FastSpeech2/preprocessed_data/LJSpeech


In [7]:
%cd /content/FastSpeechPPG/FastSpeech2
# 需要23分钟
!python3 preprocess.py config/LJSpeech/preprocess.yaml

/content/FastSpeechPPG/FastSpeech2
Processing Data ...
100% 1/1 [21:57<00:00, 1317.37s/it]
Computing statistic quantities ...
Total time: 23.62337556059461 hours


In [5]:
#@title Train
%cd /content/FastSpeechPPG/FastSpeech2
!python3 train.py -p config/LJSpeech/preprocess.yaml -m config/LJSpeech/model.yaml -t config/LJSpeech/train.yaml

/content/FastSpeechPPG/FastSpeech2
Prepare training ...
Number of FastSpeech2 Parameters: 35159361
Removing weight norm...
2023-03-17 17:00:05.599833: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 17:00:06.550118: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-17 17:00:06.550244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory

# Train （我们之后要用的）

In [6]:
#@title 下载 librispeech
%cd /content
!wget --no-check-certificate -r https://us.openslr.org/resources/12/train-clean-100.tar.gz

/content
--2023-03-17 17:42:17--  https://us.openslr.org/resources/12/train-clean-100.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘us.openslr.org/resources/12/train-clean-100.tar.gz’

us.openslr.org/reso 100%[===================>]   5.95G  12.8MB/s    in 7m 43s  

2023-03-17 17:50:01 (13.2 MB/s) - ‘us.openslr.org/resources/12/train-clean-100.tar.gz’ saved [6387309499/6387309499]

FINISHED --2023-03-17 17:50:01--
Total wall clock time: 7m 44s
Downloaded: 1 files, 5.9G in 7m 43s (13.2 MB/s)


In [7]:
#@title 解压 librispeech
%cd /content/us.openslr.org/resources/12
import tarfile
filename = "train-clean-100.tar.gz"
tf = tarfile.open(filename)
tf.extractall('/content')

/content/us.openslr.org/resources/12


In [ ]:
#@title 预处理，算每个音频的mel
import numpy as np
from scipy.io.wavfile import read
import torch
import layers

def load_wav_to_torch(full_path):
    sampling_rate, data = read(full_path)
    return torch.FloatTensor(data.astype(np.float32)), sampling_rate

def get_mel(self, filename):
    sampling_rate, data = read(filename)
    audio = torch.FloatTensor(data.astype(np.float32))
    sampling_rate = sampling_rate
    if sampling_rate != self_sampling_rate:
        raise ValueError("{} {} SR doesn't match target {} SR".format(
            sampling_rate, self_sampling_rate))
    audio_norm = audio / max_wav_value
    audio_norm = audio_norm.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    melspec = self.stft.mel_spectrogram(audio_norm)
    melspec = torch.squeeze(melspec, 0)

    # 还需要写入规定格式名称的文件
    # "{}-mel-{}.npy".format(speaker, basename)

    return melspec

max_wav_value = 32768.0
self_sampling_rate = 22050
filter_length = 1024
hop_length = 256
win_length = 1024
n_mel_channels = 80
mel_fmin = 0.0
mel_fmax = 8000.0

stft = layers.TacotronSTFT(
        filter_length, hop_length, win_length,
        n_mel_channels, self_sampling_rate, mel_fmin,
        mel_fmax)
